# **네이버 영화 리뷰 - GRU**

구글 드라이브 마운트

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


패딩 데이터 불러오기

In [ ]:
import numpy as np

# x = data / y = label
x_train = np.load("/content/drive/MyDrive/ml_data/NaverMovieReview/x_train_padded.npy", allow_pickle=True)
y_train = np.load("/content/drive/MyDrive/ml_data/NaverMovieReview/y_train_tokenized.npy")
x_test = np.load("/content/drive/MyDrive/ml_data/NaverMovieReview/x_test_padded.npy", allow_pickle=True)
y_test = np.load("/content/drive/MyDrive/ml_data/NaverMovieReview/y_test_tokenized.npy")

모델 생성

In [ ]:
from tensorflow.keras.layers import Embedding, Dense, GRU
from tensorflow.keras.models import Sequential

# 정수 인코딩을 통해서 구한 사이즈
vocab_size = 28631 # = 단어 개수 (colab '네이버 영화 리뷰 - 정수 인코딩' 참고)

model = Sequential()
model.add(Embedding(vocab_size, 100))
model.add(GRU(128))
model.add(Dense(1, activation="sigmoid"))

모델 컴파일

In [ ]:
model.compile(
    optimizer="adam", 
    loss="binary_crossentropy", 
    metrics=["acc"]
    )

모델 훈련

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# EarlyStopping, 과적합으로 손실이 4회이상 발생하면 조기 종료
es = EarlyStopping(
    monitor="val_loss", 
    mode="min", 
    verbose=1, 
    patience=4
    )

# ModelCheckpoint, 검증 데이터의 정확도가 이전보다 좋아질때만 모델을 저장
mc = ModelCheckpoint(
    "/content/drive/MyDrive/ml_data/models/NaverMovieReview/best_model_GRU.h5", 
    monitor="val_acc", 
    mode="max", 
    verbose=1, 
    save_best_only=True)

history = model.fit(
    x_train, y_train,
    epochs=20,
    batch_size=100,
    callbacks=[es, mc],
    validation_data=(x_test, y_test),
    verbose=1
)

Epoch 1/20
1458/1458 [==============================] - 264s 179ms/step - loss: 0.3716 - acc: 0.8329 - val_loss: 0.3301 - val_acc: 0.8547

Epoch 00001: val_acc improved from -inf to 0.85474, saving model to /content/drive/MyDrive/ml_data/models/NaverMovieReview/best_model_GRU.h5
Epoch 2/20
1458/1458 [==============================] - 256s 176ms/step - loss: 0.2762 - acc: 0.8846 - val_loss: 0.3275 - val_acc: 0.8560

Epoch 00002: val_acc improved from 0.85474 to 0.85596, saving model to /content/drive/MyDrive/ml_data/models/NaverMovieReview/best_model_GRU.h5
Epoch 3/20
1458/1458 [==============================] - 256s 175ms/step - loss: 0.2247 - acc: 0.9086 - val_loss: 0.3464 - val_acc: 0.8562

Epoch 00003: val_acc improved from 0.85596 to 0.85617, saving model to /content/drive/MyDrive/ml_data/models/NaverMovieReview/best_model_GRU.h5
Epoch 4/20
1458/1458 [==============================] - 254s 174ms/step - loss: 0.1811 - acc: 0.9289 - val_loss: 0.3769 - val_acc: 0.8510

Epoch 00004: va

모델 평가

In [ ]:
loss, acc = model.evaluate(x_test, y_test)
print('정확도: {:5.2f}%'.format(100 * acc))

1532/1532 [==============================] - 9s 6ms/step - loss: 0.8108 - acc: 0.8345
정확도: 83.45%
